In [ ]:
NU = 6

In [ ]:
import numpy as np
import pandas as pd
import os

dir_sim_ = './simulations'
dir_tr_input_ = './transformed_input'

udl_distrib = pd.read_csv(os.path.join(dir_sim_, 'nu_eq_%i.csv' % NU), header=0, index_col=0)
members_pos = pd.read_csv(os.path.join(dir_tr_input_, 'positions.csv'), header=0, index_col=0)

loss_and_profit = np.dot(members_pos.values, udl_distrib.values).transpose()

print loss_and_profit.shape

In [ ]:
from lib.msra_loss import MSRALossFunctionAbs

In [ ]:
class LossFunction1(MSRALossFunctionAbs):
    def shortfall_risk(self, m=None):
        m = self._check_argument(m)
        x_minus_m = np.subtract(self.x, m)
        
        sum_x_minus_m_plus = np.maximum(x_minus_m, 0.).sum(axis=1)
        sum_x_minus_m_minus = -np.minimum(x_minus_m, 0.).sum(axis=1)

        diff = sum_x_minus_m_plus - 0.5 * sum_x_minus_m_minus
        return diff.mean()

    def shortfall_risk_jac(self, m):
        m = self._check_argument(m)
        x_minus_m = np.subtract(self.x, m)
        
        sgn = np.sign(x_minus_m)
        
        sgn_plus = np.maximum(sgn, 0.)
        sgn_minus = -np.minimum(sgn, 0.)
        
        diff = sgn_plus + 0.5 * sgn_minus
        
        return diff.mean(axis=0)

In [ ]:
loss_obj = LossFunction1(loss_and_profit, 0.)

In [ ]:
RESULTS = pd.DataFrame(index=members_pos.index)

In [ ]:
from scipy.optimize import minimize

methods = ['SLSQP']
maxiter = 10000

In [ ]:
guess = np.maximum(np.amax(loss_and_profit, axis=0), 0.)

cont = loss_obj.ineq_constraint(guess) >= 0.
while cont:
    guess = 0.99 * guess
    tmp = loss_obj.ineq_constraint(guess)
    cont = tmp >= 0.

print loss_obj.ineq_constraint(guess)

In [ ]:
cons = ({'type': 'ineq',
         'fun' : lambda x: loss_obj.ineq_constraint(x),
         'jac' : lambda x: loss_obj.ineq_constraint_jac(x)})
    
bounds = [(0, None) for _ in xrange(loss_obj.dim)]

res = minimize(loss_obj.objective, guess, 
               jac=loss_obj.objective_jac, 
               constraints=cons, 
               method='SLSQP',
               bounds=bounds,
               options={'maxiter': maxiter, 'disp': True})

RESULTS[r'$\nu = %i$ bounds' % NU] = res.x
print loss_obj.ineq_constraint(res.x)

In [ ]:
dir_ = './results'

if not os.path.exists(dir_):
    os.makedirs(dir_)

In [ ]:
RESULTS.to_csv(os.path.join(dir_, 'l1_nu_eq_%i.csv' % NU))

In [ ]:
RESULTS